In [1]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from collections import Counter
import numpy as np
from collections import defaultdict
import json
from pathlib import Path
from langchain.prompts import PromptTemplate
import logging
import time
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
import torch
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
from heapq import nlargest
import torch.nn as nn
from transformers import set_seed


random_seed = 123
#The link of the files is https://github.com/salesforce/QAConv/tree/master/dataset, which in QAConv-V1.0.zip

QA = pd.read_json("E:/hotpot/hotpot_dev_distractor_v1_sample.json") #For the file here, either trn.json, tst.json or val.json can be used as Question_Answer data
QA['text']=''


In [36]:
QA[:1]['context'][0][0][1]


['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
 " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
 ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']

In [48]:
for i in range(len(QA)):
    text=''
    for j in (QA.iloc[i]['context']):
        for k in j[1]:
            text+=k
    QA.loc[i,'text']=text

In [53]:
QA.to_json('E:/hotpot/hotpot_sample.json', orient='records')

In [54]:
QA

,_id,answer,question,supporting_facts,context,type,level,text
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Ed Wood (film), [Ed Wood is a 1994 American ...",comparison,hard,Ed Wood is a 1994 American biographical period...
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[Meet Corliss Archer, [Meet Corliss Archer, a...",bridge,hard,"Meet Corliss Archer, a program from radio's Go..."
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...","[[Andre Norton Award, [The Andre Norton Award ...",bridge,hard,The Andre Norton Award for Young Adult Science...
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]","[[Esma Sultan (daughter of Abdülaziz), [Esma S...",comparison,hard,Esma Sultan (21 March 1873 – 7 May 1899) was a...
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...","[[Just Another Romantic Wrestling Comedy, [Jus...",bridge,hard,Just Another Romantic Wrestling Comedy is a 20...
5,5abd94525542992ac4f382d2,YG Entertainment,2014 S/S is the debut album of a South Korean ...,"[[2014 S/S, 0], [Winner (band), 0]]",[[List of awards and nominations received by S...,bridge,hard,South Korean boy group Shinee have received se...
6,5a85b2d95542997b5ce40028,Eenasul Fateh,Who was known by his stage name Aladin and hel...,"[[Eenasul Fateh, 0], [Management consulting, 0]]","[[James P. Comer, [James P. Comer (born James ...",bridge,hard,"James P. Comer (born James Pierpont Comer, Sep..."
7,5a87ab905542996e4f3088c1,"3,677 seated",The arena where the Lewiston Maineiacs played ...,"[[Lewiston Maineiacs, 1], [Androscoggin Bank C...","[[Billings Bulls, [The Billings Bulls were a j...",bridge,hard,The Billings Bulls were a junior ice hockey or...
8,5a7bbb64554299042af8f7cc,Terry Richardson,"Who is older, Annie Morton or Terry Richardson?","[[Annie Morton, 0], [Annie Morton, 2], [Terry ...","[[Annie Morton, [Annie Morton (born October 8,...",bridge,hard,"Annie Morton (born October 8, 1970) is an Amer..."
9,5a8db19d5542994ba4e3dd00,yes,Are Local H and For Against both from the Unit...,"[[Local H, 0], [For Against, 0]]","[[Mendocino County GMO Ban, [Mendocino County,...",comparison,hard,"Mendocino County, California, was the first ju..."


In [57]:
file_path='E:/hotpot/hotpot_sample.json'
combined_data = json.loads(Path(file_path).read_text())

from nltk.tokenize import word_tokenize
import collections

def compute_f1(a_gold, a_pred):
    gold_toks = word_tokenize(a_gold)
    pred_toks = word_tokenize(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def compute_precision(a_gold, a_pred):
    gold_toks = word_tokenize(a_gold)
    pred_toks = word_tokenize(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision

def compute_recall(a_gold, a_pred):
    gold_toks = word_tokenize(a_gold)
    pred_toks = word_tokenize(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return recall

def normalize_and_stem(text):
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text.lower())  # Normalize to lowercase and tokenize
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in string.punctuation and token not in stopwords.words('english')]  # Stemming and removing punctuation
    return ' '.join(stemmed_tokens)

#Modified template
template =  """
            Based on the following information only: 
                    
            {context}
                    
            {question} Please provide the answer in as few words as possible and please do NOT repeat any word in the question, i.e. "{question}".

            """

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
#llm = GPT4All(model="E:/orca-mini-3b-gguf2-q4_0.gguf", max_tokens=2048, seed=random_seed)
llm = GPT4All(model="E:/mistral-7b-instruct-v0.1.Q4_0.gguf", max_tokens=2048, seed=random_seed)

instruct_embedding_model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruct_embedding_model_kwargs = {'device': 'cpu'}
instruct_embedding_encode_kwargs = {'normalize_embeddings': True}

answers_pairs=[]
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
top_n_sentences = [1, 2]
dist_functions = ['pairwise', 'cosine']


def process_story_questions(dist_functions,top_n_sentences,combined_data, model_name, instruction,chunk_size,overlap_percentage):
    hf_story_embs = HuggingFaceInstructEmbeddings(
        model_name=instruct_embedding_model_name,
        model_kwargs=instruct_embedding_model_kwargs,
        encode_kwargs=instruct_embedding_encode_kwargs,
    )
    hf_query_embs = HuggingFaceInstructEmbeddings(
        model_name=instruct_embedding_model_name,
        model_kwargs=instruct_embedding_model_kwargs,
        encode_kwargs=instruct_embedding_encode_kwargs,
        query_instruction="How does this information relate to the question?"
    )
    for func in dist_functions:
        print(f"\nProcessing distance function: {func}")
        for top_n_sentence in top_n_sentences:
            print(f"\nTop sentences number: [{top_n_sentence}]")
            for cov in combined_data:
                text_splitter = RecursiveCharacterTextSplitter()
                sentences = sent_tokenize(cov['text'])
                sentence_embs = hf_story_embs.embed_documents(sentences)
                all_splits = text_splitter.split_text(cov['text'])
                vectorstore = Chroma.from_texts(texts=all_splits, embedding=hf_story_embs)

                question = cov['question']
                question_emb = hf_query_embs.embed_query(question)
                if dist_functions == 'pairwise':
                    scores = [pdist(torch.tensor(sentence_emb).unsqueeze(0), torch.tensor(question_emb).unsqueeze(0)).item() for sentence_emb in sentence_embs]
                else:
                    scores = [torch.cosine_similarity(torch.tensor(sentence_emb).unsqueeze(0), torch.tensor(question_emb).unsqueeze(0))[0].item() for sentence_emb in sentence_embs]
                top_scores_indices = nlargest(top_n_sentence, range(len(scores)), key=lambda idx: scores[idx])
                context_for_qa = " ".join([sentences[idx] for idx in top_scores_indices])
                qa_chain = RetrievalQA.from_chain_type(
                            llm, 
                            retriever=vectorstore.as_retriever(search_kwargs={'k': 1}), 
                            chain_type="stuff",
                            verbose=True,
                            chain_type_kwargs={
                                "prompt": QA_CHAIN_PROMPT,
                                "verbose": True},
                            return_source_documents=False)
                #if (len(context_tokens)>2048):
                #    context_tokens=context_tokens[0:2047]
                #context_for_qa = " ".join(context_for_qa[i] for i in range(len(context_tokens)))
                inputs = qa_chain.prep_inputs({"query": question})

                #predict = qa_chain({"query": question})
                print(context_for_qa)
                predict=qa_chain({"context": context_for_qa, "query": question})
                #if predict['result']=='error prompt size exce context window size process':
                #    context_for_qa=context_for_qa
                #else:
                #    context_for_qa=''
                answers_pairs.append((cov['answer'],predict,chunk_size,overlap_percentage,top_n_sentence,func,context_for_qa))
                del qa_chain
                del vectorstore
                del all_splits
            del text_splitter

In [58]:
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruction = "Represent the story for retrieval:" 
#Try different chunk sizes and overlap percentages
#chunk_sizes = [100, 200]
#overlap_percentages = [0, 0.1]
#for i in chunk_sizes:
#    for j in overlap_percentages:
#        text_splitter = RecursiveCharacterTextSplitter(chunk_size=i, chunk_overlap=j)
#        process_story_questions(combined_data, model_name, instruction, i, j)
chunk_size=200
overlap_percentage=0.1
pdist = nn.PairwiseDistance(p=2.0, eps=1e-06)
top_n_sentences = [1]
dist_functions = ['cosine']
process_story_questions(dist_functions,top_n_sentences,combined_data, model_name, instruction, chunk_size, overlap_percentage)
result_list=[]
for i in answers_pairs:
    actual_answer=normalize_and_stem(i[0])
    predict_answer=normalize_and_stem(i[1]['result'])
    f1_score = compute_f1(actual_answer,predict_answer)
    precision = compute_precision(actual_answer,predict_answer)
    recall = compute_recall(actual_answer,predict_answer)

    #print(f"Question: {i[0]}")
    #print(f"Predicted Answer: {i[2]}")
    #print(f"Actual Answer: {i[1][0]}")
    #print(f"F1 Score: {f1_score:.4f}")
    #print(f"Precision: {precision:.4f}")
    #print(f"Recall: {recall:.4f}")
    #print()
    result_list.append([i[6],i[4],i[5],i[2],i[3],i[1]['query'],predict_answer,actual_answer,f1_score,precision,recall])

df=pd.DataFrame({'Context':[], 'top_n_sentences':[],'dist_function':[],'Chunk_size':[],'Overlap_percentage':[],'Quesion':[],'Predicted Answer':[],'Actual Answer':[],'f1 score':[],'Precision':[],'Recall':[]})
while((len(df))!=(len(result_list))):
    df.loc[len(df)]=result_list[len(df)]
df.to_csv('output_hotpot.csv', index=False)

load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512

Processing distance function: pairwise

Top sentences number: [1]
Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and businessman at the turn of the 20th century.


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

            Based on the following information only: 
                    
            Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood. The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau. Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.Sco